In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
main_url = 'https://www.newsbomb.gr/'

start_url = 'https://www.newsbomb.gr/tag/syriza?page='

start_page = 1
end_page = 6

In [ ]:
response = requests.get(start_url)
doc = BeautifulSoup(response.text, 'html.parser')

In [ ]:
teasers_list = []

for page_num in range(start_page, end_page+1):
  page_url = start_url + str(page_num)
  response = requests.get(page_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  teaser_articles_list = doc.find_all('div', {'class': 'card-item'})


  for teaser in teaser_articles_list:
      story_dict = {}
      try:

        link = teaser.find('a')['href']


        if link.startswith("http"):
          story_dict['url'] = link
        else:
          story_dict['url'] = main_url + link
      except:
        story_dict['url'] = ''
        pass
      teasers_list.append(story_dict)
  time.sleep(1)

In [ ]:
bomb_teasers_df = pd.DataFrame(teasers_list)
bomb_teasers_df

,url
0,https://www.newsbomb.gr//politikh/story/144805...
1,https://www.newsbomb.gr//politikh/story/144797...
2,https://www.newsbomb.gr//politikh/story/144774...
3,https://www.newsbomb.gr//politikh/story/144769...
4,https://www.newsbomb.gr//politikh/story/144768...
...,...
145,https://www.newsbomb.gr//me-to-ni-me-to-sigma/...
146,https://www.newsbomb.gr//meeting-point-olga-tr...
147,https://www.newsbomb.gr//politikh/story/144324...
148,https://www.newsbomb.gr//politikh/story/144324...


In [ ]:
article_url = 'https://www.newsbomb.gr/politikh/story/1447297/syriza-i-axtsioglou-anakoinonei-ypopsifiotita-o-pappas-tis-teleftaies-kiniseis'

In [ ]:
response = requests.get(article_url)
doc = BeautifulSoup(response.text, 'html.parser')

In [ ]:
full_articles_list = []

for article_url in bomb_teasers_df['url']:
  response = requests.get(article_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  article = doc.find('section', {'class': 'article__main'})

  full_article_dict = {}

  full_article_dict['site'] = "https://www.newsbomb.gr/"

  full_article_dict['url'] = article_url

  try:
    section = doc.find('a', {'class': 'main-category'}).text
    full_article_dict['section'] = section
  except:
    full_article_dict['section'] = ''
    pass

  try:
    title = doc.find('h1').text
    full_article_dict['title'] = title
  except:
    full_article_dict['title'] = ''
    pass

  try:
    date = doc.find('time')['datetime']
    full_article_dict['date'] = date
  except:
    full_article_dict['date'] = ''
    pass

  try:
    author = doc.find('p', {'class', "main-author story-author"}).span.text
    full_article_dict['author'] = author
  except:
    full_article_dict['author'] = ''
    pass

  try:
    p_texts_list = []
    paragraphs = doc.find_all('p')
    for p in paragraphs:
      p_texts_list.append(p.text)
    full_text = ' '.join(p_texts_list)
    full_text = "".join(full_text.splitlines())
    full_article_dict['full_text'] = full_text
  except:
    full_article_dict['full_text'] = ''
    pass

  full_articles_list.append(full_article_dict)
  time.sleep(1)


bomb_full_articles_df = pd.DataFrame(full_articles_list)
bomb_full_articles_df

,site,url,section,title,date,author,full_text
0,https://www.newsbomb.gr/,https://www.newsbomb.gr//politikh/story/144805...,ΠΟΛΙΤΙΚΗ,Η Λούκα Κατσέλη πλέκει το εγκώμιο της Έφης Αχτ...,2023-07-13T18:47:43+03:00,Newsbomb,Newsbomb Η Λούκα Κατσέλη με ανάρτησή της στηρί...
1,https://www.newsbomb.gr/,https://www.newsbomb.gr//politikh/story/144797...,ΠΟΛΙΤΙΚΗ,Παρασκήνιο: To διπλό άδειασμα Μητσοτάκη στον Μ...,2023-07-13T14:26:00+03:00,,Κώστας Τσιτούνας Η ιστορία με την Πανεπιστημια...
2,https://www.newsbomb.gr/,https://www.newsbomb.gr//politikh/story/144774...,ΠΟΛΙΤΙΚΗ,Τα μηνύματα της Αχτσιόγλου και η πλατιά συστρά...,2023-07-13T06:08:08+03:00,,"Αντώνης Γκιόκας Μηνύματα εντός, αλλά και εκτό..."
3,https://www.newsbomb.gr/,https://www.newsbomb.gr//politikh/story/144769...,ΠΟΛΙΤΙΚΗ,Η Αχτσιόγλου ανάβει «φωτιές» – Συντροφικά «μαχ...,2023-07-12T18:53:06+03:00,Newsbomb,Newsbomb Η σημερινή επιβεβαίωση της Έφης Αχτσι...
4,https://www.newsbomb.gr/,https://www.newsbomb.gr//politikh/story/144768...,ΠΟΛΙΤΙΚΗ,Αλέξης Χαρίτσης: Δεν θα είναι υποψήφιος για τη...,2023-07-12T18:20:31+03:00,Newsbomb,Newsbomb Ο Αλέξης Χαρίτσης δεν θα είναι υποψήφ...
...,...,...,...,...,...,...,...
145,https://www.newsbomb.gr/,https://www.newsbomb.gr//me-to-ni-me-to-sigma/...,ME TO N & ME TO Σ,"Τι παίρνει, δυστυχώς, μαζί του ο Αλέξης Τσίπρας…",2023-06-29T16:13:58+03:00,,Νίκος Συρίγος Ο ΣΥΡΙΖΑ με την αποχώρηση του Τσ...
146,https://www.newsbomb.gr/,https://www.newsbomb.gr//meeting-point-olga-tr...,MEETING POINT,Γενναία απόφαση Τσίπρα - Πλεύση ΣΥΡΙΖΑ σε αχαρ...,2023-06-29T16:13:05+03:00,,Όλγα Τρέμη Τέλος εποχής για τον ΣΥΡΙΖΑ. Η απόφ...
147,https://www.newsbomb.gr/,https://www.newsbomb.gr//politikh/story/144324...,ΠΟΛΙΤΙΚΗ,ΣΥΡΙΖΑ: «Αποχώρησε μετά τη βαριά ήττα στις εκλ...,2023-06-29T16:03:46+03:00,Newsbomb,Newsbomb Οι εξελίξεις στον ΣΥΡΙΖΑ βρέθηκαν στο...
148,https://www.newsbomb.gr/,https://www.newsbomb.gr//politikh/story/144324...,ΠΟΛΙΤΙΚΗ,Διαψεύδουν πηγές του ΣΥΡΙΖΑ τα περί παραίτησης...,2023-06-29T15:47:25+03:00,Newsbomb,Newsbomb Άνω κάτω στον ΣΥΡΙΖΑ με τις παραιτήσε...


In [ ]:
bomb_full_articles_df['clean_text'] = bomb_full_articles_df['full_text'].str.replace("Newsbomb", "")

In [ ]:
bomb_full_articles_df['datetime'] = pd.to_datetime(bomb_full_articles_df['date'], format='%Y-%m-%d %H:%M')
bomb_full_articles_df.head()

,site,url,section,title,date,author,full_text,clean_text,datetime
0,https://www.newsbomb.gr/,https://www.newsbomb.gr//politikh/story/144805...,ΠΟΛΙΤΙΚΗ,Η Λούκα Κατσέλη πλέκει το εγκώμιο της Έφης Αχτ...,2023-07-13T18:47:43+03:00,Newsbomb,Newsbomb Η Λούκα Κατσέλη με ανάρτησή της στηρί...,Η Λούκα Κατσέλη με ανάρτησή της στηρίζει σαφώ...,2023-07-13 18:47:43+03:00
1,https://www.newsbomb.gr/,https://www.newsbomb.gr//politikh/story/144797...,ΠΟΛΙΤΙΚΗ,Παρασκήνιο: To διπλό άδειασμα Μητσοτάκη στον Μ...,2023-07-13T14:26:00+03:00,,Κώστας Τσιτούνας Η ιστορία με την Πανεπιστημια...,Κώστας Τσιτούνας Η ιστορία με την Πανεπιστημια...,2023-07-13 14:26:00+03:00
2,https://www.newsbomb.gr/,https://www.newsbomb.gr//politikh/story/144774...,ΠΟΛΙΤΙΚΗ,Τα μηνύματα της Αχτσιόγλου και η πλατιά συστρά...,2023-07-13T06:08:08+03:00,,"Αντώνης Γκιόκας Μηνύματα εντός, αλλά και εκτό...","Αντώνης Γκιόκας Μηνύματα εντός, αλλά και εκτό...",2023-07-13 06:08:08+03:00
3,https://www.newsbomb.gr/,https://www.newsbomb.gr//politikh/story/144769...,ΠΟΛΙΤΙΚΗ,Η Αχτσιόγλου ανάβει «φωτιές» – Συντροφικά «μαχ...,2023-07-12T18:53:06+03:00,Newsbomb,Newsbomb Η σημερινή επιβεβαίωση της Έφης Αχτσι...,Η σημερινή επιβεβαίωση της Έφης Αχτσιόγλου ότ...,2023-07-12 18:53:06+03:00
4,https://www.newsbomb.gr/,https://www.newsbomb.gr//politikh/story/144768...,ΠΟΛΙΤΙΚΗ,Αλέξης Χαρίτσης: Δεν θα είναι υποψήφιος για τη...,2023-07-12T18:20:31+03:00,Newsbomb,Newsbomb Ο Αλέξης Χαρίτσης δεν θα είναι υποψήφ...,Ο Αλέξης Χαρίτσης δεν θα είναι υποψήφιος για ...,2023-07-12 18:20:31+03:00


In [ ]:
bomb_full_articles_df['year'] = pd.DatetimeIndex(bomb_full_articles_df['date']).year
bomb_full_articles_df['month'] = pd.DatetimeIndex(bomb_full_articles_df['date']).month
bomb_full_articles_df['day'] = pd.DatetimeIndex(bomb_full_articles_df['date']).day
bomb_full_articles_df['hour'] = pd.DatetimeIndex(bomb_full_articles_df['date']).hour

In [ ]:
bomb_full_articles_df['datetime'] = bomb_full_articles_df['datetime'].dt.tz_convert(None)

In [ ]:
bomb_full_articles_df.dtypes

site                  object
url                   object
section               object
title                 object
date                  object
author                object
full_text             object
clean_text            object
datetime      datetime64[ns]
year                   int64
month                  int64
day                    int64
hour                   int64
dtype: object

In [ ]:
bomb_full_articles_df.head()

,site,url,section,title,date,author,full_text,clean_text,datetime,year,month,day,hour
0,https://www.newsbomb.gr/,https://www.newsbomb.gr//politikh/story/144805...,ΠΟΛΙΤΙΚΗ,Η Λούκα Κατσέλη πλέκει το εγκώμιο της Έφης Αχτ...,2023-07-13T18:47:43+03:00,Newsbomb,Newsbomb Η Λούκα Κατσέλη με ανάρτησή της στηρί...,Η Λούκα Κατσέλη με ανάρτησή της στηρίζει σαφώ...,2023-07-13 15:47:43,2023,7,13,18
1,https://www.newsbomb.gr/,https://www.newsbomb.gr//politikh/story/144797...,ΠΟΛΙΤΙΚΗ,Παρασκήνιο: To διπλό άδειασμα Μητσοτάκη στον Μ...,2023-07-13T14:26:00+03:00,,Κώστας Τσιτούνας Η ιστορία με την Πανεπιστημια...,Κώστας Τσιτούνας Η ιστορία με την Πανεπιστημια...,2023-07-13 11:26:00,2023,7,13,14
2,https://www.newsbomb.gr/,https://www.newsbomb.gr//politikh/story/144774...,ΠΟΛΙΤΙΚΗ,Τα μηνύματα της Αχτσιόγλου και η πλατιά συστρά...,2023-07-13T06:08:08+03:00,,"Αντώνης Γκιόκας Μηνύματα εντός, αλλά και εκτό...","Αντώνης Γκιόκας Μηνύματα εντός, αλλά και εκτό...",2023-07-13 03:08:08,2023,7,13,6
3,https://www.newsbomb.gr/,https://www.newsbomb.gr//politikh/story/144769...,ΠΟΛΙΤΙΚΗ,Η Αχτσιόγλου ανάβει «φωτιές» – Συντροφικά «μαχ...,2023-07-12T18:53:06+03:00,Newsbomb,Newsbomb Η σημερινή επιβεβαίωση της Έφης Αχτσι...,Η σημερινή επιβεβαίωση της Έφης Αχτσιόγλου ότ...,2023-07-12 15:53:06,2023,7,12,18
4,https://www.newsbomb.gr/,https://www.newsbomb.gr//politikh/story/144768...,ΠΟΛΙΤΙΚΗ,Αλέξης Χαρίτσης: Δεν θα είναι υποψήφιος για τη...,2023-07-12T18:20:31+03:00,Newsbomb,Newsbomb Ο Αλέξης Χαρίτσης δεν θα είναι υποψήφ...,Ο Αλέξης Χαρίτσης δεν θα είναι υποψήφιος για ...,2023-07-12 15:20:31,2023,7,12,18


In [ ]:
bomb_full_articles_df.to_csv('/content/gdrive/MyDrive/DATA_JOUR/FINAL ESSAY/bomb_full_df.csv')